In [1]:
install.packages("Distance")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
install.packages("dsm")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [3]:
library(Distance)
library(dsm)

Loading required package: mrds

This is mrds 3.0.0
Built: R 4.3.3; ; 2025-06-25 15:01:20 UTC; unix


Attaching package: ‘Distance’


The following object is masked from ‘package:mrds’:

    create.bins


Loading required package: mgcv

Loading required package: nlme

This is mgcv 1.9-3. For overview type 'help("mgcv-package")'.

Loading required package: numDeriv

Registered S3 methods overwritten by 'dsm':
  method                 from    
  predict.fake_ddf       Distance
  print.fake_ddf         Distance
  print.summary.fake_ddf Distance
  summary.fake_ddf       Distance

This is dsm 2.3.3
Built: R 4.3.3; ; 2025-06-26 10:18:17 UTC; unix


Attaching package: ‘dsm’


The following object is masked from ‘package:Distance’:

    dummy_ddf




In [4]:
data_dir <- "../../../spermwhaledata"
csv_data_dir <- paste(data_dir, "csv", sep="/")
generated_data_dir <- paste(data_dir, "generated", sep="/")
dir.create(generated_data_dir)

In [5]:
dist <- read.csv(paste(csv_data_dir, "distdata.csv", sep="/"))
obs <- read.csv(paste(csv_data_dir, "obs.csv", sep="/"))
predgrid <- read.csv(paste(csv_data_dir, "predgrid.csv", sep="/"))
segs <- read.csv(paste(csv_data_dir, "segs.csv", sep="/"))
head(dist, 3)

,Survey,SightingTime,distance,object,Sample.Label,size,detected,observer,Beaufort,long,lat,x,y
,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,en04395,2004-06-28 11:22:21,246.0173,1,48,2,1,1,3.0,-65.636,39.576,633725.6,642049.0
2,en04395,2004-06-28 14:18:14,1632.3934,2,50,2,1,1,2.5,-65.648,39.746,631348.3,660752.3
3,en04395,2004-06-28 15:13:34,2368.9941,3,51,1,1,1,3.0,-65.692,39.843,626812.7,671194.6


In [6]:
df_hn <- ds(data=dist, truncation=6000, key="hn", adjustment=NULL)
df_hn

Fitting half-normal key function

AIC= 2252.06

No survey area information supplied, only estimating detection function.





Distance sampling analysis object

Detection function:
 Half-normal key function 

Estimated abundance in covered region: 240.416 

In [7]:
my_dsm <- dsm(count~s(x,y, bs="ts") +
                    s(Depth, bs="ts"),
              df_hn, segs, obs,
              family=tw())
my_dsm

Warning message in make.data(response, ddf.obj, segment.data, observation.data, :
“Some observations are outside of detection function 1 truncation!”



Density surface model
Response :  count 

Detection function : Half-normal key function 

Formula:  count ~ s(x, y, bs = "ts") + s(Depth, bs = "ts") + offset(off.set) 

Fitting engine: gam


In [8]:
predgrid$density <- predict(my_dsm, newdata=predgrid, offset=predgrid$off.set)
head(predgrid)

,x,y,Depth,SST,NPP,DistToCAS,EKE,off.set,long,lat,density
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl[1d]>
1,547984.6,788254,153.59825,12.04609,1462.521,11788.974,0.0008329031,1e+08,-66.52252,40.94697,0.006334498
2,557984.6,788254,552.31067,12.81379,1465.410,5697.248,0.0009806611,1e+08,-66.40464,40.94121,0.023877901
3,527984.6,778254,96.81992,12.90251,1429.432,13722.626,0.0011575423,1e+08,-66.76551,40.86781,0.005421990
4,537984.6,778254,138.23763,13.21393,1424.862,9720.671,0.0013417297,1e+08,-66.64772,40.86227,0.005965414
5,547984.6,778254,505.14386,13.75655,1379.351,8018.690,0.0026881567,1e+08,-66.52996,40.85662,0.020205362
6,557984.6,778254,1317.59521,14.42525,1348.544,3775.462,0.0045683752,1e+08,-66.41221,40.85087,0.263562208


In [9]:
write.csv(predgrid, paste(generated_data_dir, "predgrid.csv", sep="/"), row.names = FALSE)

In [10]:
predgrid_d <- predgrid
predgrid_d$densityd <- cut(predgrid_d$density, c(0, 0.1, 0.5, 1, 2, 3.1))
head(predgrid_d)

,x,y,Depth,SST,NPP,DistToCAS,EKE,off.set,long,lat,density,densityd
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl[1d]>,<fct>
1,547984.6,788254,153.59825,12.04609,1462.521,11788.974,0.0008329031,1e+08,-66.52252,40.94697,0.006334498,"(0,0.1]"
2,557984.6,788254,552.31067,12.81379,1465.410,5697.248,0.0009806611,1e+08,-66.40464,40.94121,0.023877901,"(0,0.1]"
3,527984.6,778254,96.81992,12.90251,1429.432,13722.626,0.0011575423,1e+08,-66.76551,40.86781,0.005421990,"(0,0.1]"
4,537984.6,778254,138.23763,13.21393,1424.862,9720.671,0.0013417297,1e+08,-66.64772,40.86227,0.005965414,"(0,0.1]"
5,547984.6,778254,505.14386,13.75655,1379.351,8018.690,0.0026881567,1e+08,-66.52996,40.85662,0.020205362,"(0,0.1]"
6,557984.6,778254,1317.59521,14.42525,1348.544,3775.462,0.0045683752,1e+08,-66.41221,40.85087,0.263562208,"(0.1,0.5]"


In [11]:
write.csv(predgrid_d, paste(generated_data_dir, "predgrid_d.csv", sep="/"), row.names = FALSE)

In [12]:
# sample from a poisson with intensity==density
ndot <- rpois(rep(1,nrow(predgrid)), predgrid$density)
# make a new data set with locations of "observations" above
# some of the predgrid gets dropped, the rest multiplied
rep_grid <- predgrid[rep(1:nrow(predgrid), ndot), ]
head(rep_grid)

,x,y,Depth,SST,NPP,DistToCAS,EKE,off.set,long,lat,density
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl[1d]>
13,557984.6,768254,1829.363,16.28555,1250.005,4541.3862,0.01737181,1e+08,-66.41976,40.76056,0.6347843
22,547984.6,758254,1826.542,17.78271,1260.608,2561.7629,0.02221215,1e+08,-66.54478,40.67599,0.6290825
23,557984.6,758254,2139.372,18.43224,1213.327,1949.2075,0.02980540,1e+08,-66.42730,40.67026,0.6959115
23.1,557984.6,758254,2139.372,18.43224,1213.327,1949.2075,0.02980540,1e+08,-66.42730,40.67026,0.6959115
24,567984.6,758254,2337.434,18.95543,1200.634,650.7568,0.03842679,1e+08,-66.30983,40.66442,0.6472542
25,577984.6,758254,2704.084,19.40874,1216.026,1194.6709,0.04799516,1e+08,-66.19239,40.65848,0.5266832


In [13]:
write.csv(rep_grid, paste(generated_data_dir, "rep_grid.csv", sep="/"), row.names = FALSE)